In [1]:
import torch

In [ ]:
action_slc_n_epochs = 100
action_slc_lr = 0.1
belief_updt_n_epochs = 100
belief_updt_lr = 0.1
n_target = 2
mvt_amplitude = 3
max_coord = torch.tensor([300., 600.])
n_step_per_rollout = 1
n_spl_epist_val = 5

In [ ]:
a = torch.nn.Parameter(torch.zeros(n_target))

opt = torch.optim.Adam([a, ], lr=belief_updt_lr)

for step in range(action_slc_n_epochs):

    old_a = a.clone()
    opt.zero_grad()

    action = torch.sigmoid(a)*360
    total_efe = 0
    for _ in range(n_rollout):
        efe_rollout = 0

        x_rollout = x.clone()
        b_rollout = b.clone()

        action_plan = torch.zeros((n_step_per_rollout, n_target))
        action_plan[0] = action
        if n_step_per_rollout > 1:
            action_plan[1:] = torch.rand((n_step_per_rollout-1, n_target)) * 360

        for step in range(n_step_per_rollout):

            a = action_plan[step]
            prev_x = x.clone()

            pos = x
            for i in range(n_target):
                angle = a[i]
                x_prime = 1.0
                if 90 < angle <= 270:
                    x_prime *= -1

                y_prime = torch.tan(torch.deg2rad(angle)) * x_prime

                norm = mvt_amplitude / torch.sqrt(y_prime**2 + x_prime**2)
                movement = torch.tensor([x_prime, y_prime]) * norm
                pos[i] += movement

            for coord in range(2):
                for target in range(self.n_target):
                    if pos[target, coord] > max_coord[coord]:
                        pos[target, coord] = max_coord[coord]

            epistemic_value = 0
            b_prime_spl = torch.zeros((n_spl_epist_val, n_target))
            # Not sure about from which b to sample from?
            goals = torch.multinomial(torch.softmax(b, dim=0), n_spl_epist_val, replacement=True)
            for i in range(n_spl_epist_val):
                goal = goals[i]
                y = user_sim_act(goal=goal, positions=x, prev_positions=prev_x)
                
                logp_y = user_logp_action(positions=x, action=y)

                logq = torch.log_softmax(b - b.max(), dim=0)
                logp_yq = logq + logp_y

                # --- belief update --- #
                
                b_prime = torch.nn.Parameter(b.clone())
                opt = torch.optim.Adam([b_prime, ], lr=0.1)

                for step in range(belief_updt_lr):

                    old_b_prime = b_prime.clone()

                    opt.zero_grad()

                    b_prime_scaled = b_prime.clone()
                    b_prime_scaled -= b_prime_scaled.max() 
                    q_prime = torch.softmax(b_prime_scaled, dim=0)
                    kl_div = torch.sum(q_prime * (q_prime.log() - logp_yq))
                    kl_div.backward(retain_graph=True)
                    opt.step()

                    if torch.isclose(old_b_prime, b_prime).all():
                        break
                
                # ------------------- #
                
                b_prime_spl[i] = b_prime
                epistemic_value += kl_div

            epistemic_value /= n_spl_epist_val
            b_rollout = b_prime_spl[torch.randint(n_spl_epist_val, (1, ))].squeeze()
            
            efe_step = - epistemic_value  # Need to add intrinsic value

            efe_rollout += decay_factor**step * efe_step

        total_efe += efe_rollout

    total_efe /= n_rollout
    total_efe.backward(retain_graph=True)
    opt.step()

    if torch.isclose(old_a, a).all():
        break